# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [1]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [8]:
try:
    loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip')
except Exception as e:
    print("Error al cargar el archivo:", e)
    
#loans.to_csv('loans_complete.csv', index=False)
#esto carga todo a un archivo csv, lo hice para verificar que se subio correctamente, despues lo utilice para 
#checar si los types me latian


/var/folders/xy/k1hx31l15bq5npxk8h1gcpwc0000gn/T/ipykernel_4519/3786556935.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip')


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [5]:
# column_types =

data_types = pd.DataFrame(loans.dtypes).reset_index()
data_types.columns = ['column_name', 'type']
data_types
#data_types.to_csv('data_types.txt', sep='\t', index=False)

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [13]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']
datos_dict.to_csv('datos_dict_complete.csv', index=False)


In [10]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [11]:
# COdigo guardar
datos_dict = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']

# Guardar el DataFrame en formato pickle
datos_dict.to_pickle('datos_dict.pkl')

In [12]:
# Codigo para cargar
# Cargar el DataFrame desde el archivo pickle
datos_dict_loaded = pd.read_pickle('datos_dict.pkl')

# Visualizar el DataFrame cargado
print(datos_dict_loaded)

                   feature                                        description
0           acc_now_delinq  The number of accounts on which the borrower i...
1     acc_open_past_24mths         Number of trades opened in past 24 months.
2               addr_state  The state provided by the borrower in the loan...
3                 all_util              Balance to credit limit on all trades
4               annual_inc  The self-reported annual income provided by th...
..                     ...                                                ...
148      settlement_amount  The loan amount that the borrower has agreed t...
149  settlement_percentage  The settlement amount as a percentage of the p...
150        settlement_term  The number of months that the borrower will be...
151                    NaN                                                NaN
152                    NaN  * Employer Title replaces Employer Name for al...

[153 rows x 2 columns]


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [17]:
# Manejos de tipos 1

def extract_number(x):
    if pd.isnull(x):
        return 0
    if isinstance(x, str):
        return int(''.join(filter(str.isdigit, x)) or 0)
    return x

# Convertir term a int64
loans['term'] = loans['term'].apply(extract_number)

# Convertir emp_length a int64
loans['emp_length'] = loans['emp_length'].apply(extract_number)


# Convertir id y member_id a object
loans['id'] = loans['id'].astype('object')
loans['member_id'] = loans['member_id'].astype('object')
#lo cambio a object debido a que no es necesario tenerlo tipo int o float 
#ya que no se van a realizar operaciones con estos datos

# Función para extraer números de una cadena
extract_number = lambda x: int(''.join(filter(str.isdigit, x)) or 0)




# Convertir fechas a datetime
date_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 
                'last_credit_pull_d', 'hardship_start_date', 'hardship_end_date', 
                'payment_plan_start_date']

for col in date_columns:
    loans[col] = pd.to_datetime(loans[col], errors='coerce', format='%b-%Y')

# Tu codigo aqui



Las columnas como issue_d, earliest_cr_line, last_pymnt_d, entre otras, representan fechas pero están en formato de cadena. Convertirlas a datetime permite un manejo más eficiente de estas fechas en Pandas, facilitando operaciones como ordenar, filtrar por rangos de fechas o calcular intervalos de tiempo. 

La columna emp_length describe la duración del empleo en varios formatos de cadena. Para un análisis coherente y comparaciones, convertimos esta columna a un formato numérico, extrayendo el número de años de empleo como un entero. Esto también facilita cualquier análisis estadístico o agrupación basada en la duración del empleo.

La columna term originalmente contiene valores en formato de cadena, como "60 months". Sin embargo, para análisis y cálculos posteriores, es más práctico tener esta información en formato numérico. Por lo tanto, se extraen solo los dígitos numéricos y se convierten a enteros, lo que facilita operaciones como comparaciones o cálculos matemáticos relacionados con el plazo del préstamo.

Los campos id y member_id son identificadores únicos para cada préstamo y miembro, respectivamente. Estos identificadores no se utilizan para cálculos matemáticos y son más adecuados como cadenas (tipo object) para evitar confusiones, como interpretarlos como valores numéricos

In [19]:
# Definiendo la función para transformar 'emp_length'
def convert_emp_length(emp_len):
    if pd.isna(emp_len):
        return None  # Devuelve None para valores NaN
    if isinstance(emp_len, int):
        return emp_len  # Devuelve el valor si ya es un entero
    if isinstance(emp_len, str):
        if '+' in emp_len:
            # Extrae el número antes del signo '+'
            return int(emp_len.split('+')[0].strip())
        if '<' in emp_len:
            # Trata el caso '< 1 year' como 1
            return 1
        # Extrae el número de años para otros casos
        return int(''.join(filter(str.isdigit, emp_len)))
    return None  # Devuelve None para cualquier otro tipo de dato

# Aplicando la función a la columna 'emp_length'
loans['emp_length'] = loans['emp_length'].apply(convert_emp_length)



La columna emp_length representa la duración del empleo de los solicitantes de préstamos y originalmente contiene datos en varios formatos de texto, incluyendo representaciones como "10+ years" o "< 1 year". Para facilitar el análisis y permitir comparaciones numéricas eficientes

In [ ]:
# column_types =


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [26]:
# Tu codigo aqui
import json


def imputacion_eficiente(dataframe):
    imput_dict = {}
    for column_name in dataframe.columns:
        # Eliminar columnas con más del 10% de valores nulos
        if dataframe[column_name].isnull().mean() >= 0.1:
            dataframe.drop(column_name, axis=1, inplace=True)
            continue

        # Estrategias de imputación basadas en el tipo de dato
        if dataframe[column_name].dtype in ['int64', 'float64']:
            valor_imputado = dataframe[column_name].mean()
            dataframe[column_name].fillna(valor_imputado, inplace=True)
            imput_dict[column_name] = {'estrategia': 'media', 'valor': valor_imputado}
        elif dataframe[column_name].dtype in ['object', 'string']:
            valor_imputado = dataframe[column_name].mode()[0]
            dataframe[column_name].fillna(valor_imputado, inplace=True)
            imput_dict[column_name] = {'estrategia': 'moda', 'valor': valor_imputado}

    return dataframe, imput_dict

# Llamar a la función
loans, imput_dict = imputacion_eficiente(loans)
print(imput_dict)



{'Unnamed: 0': {'estrategia': 'media', 'valor': 49999.5}, 'id': {'estrategia': 'moda', 'valor': 57167}, 'loan_amnt': {'estrategia': 'media', 'valor': 14886.93}, 'funded_amnt': {'estrategia': 'media', 'valor': 14886.93}, 'funded_amnt_inv': {'estrategia': 'media', 'valor': 14883.9105}, 'term': {'estrategia': 'media', 'valor': 43.6188}, 'int_rate': {'estrategia': 'media', 'valor': 13.2780734}, 'installment': {'estrategia': 'media', 'valor': 437.3318244000001}, 'grade': {'estrategia': 'moda', 'valor': 'C'}, 'sub_grade': {'estrategia': 'moda', 'valor': 'C1'}, 'emp_title': {'estrategia': 'moda', 'valor': 'Teacher'}, 'emp_length': {'estrategia': 'media', 'valor': 5.8251}, 'home_ownership': {'estrategia': 'moda', 'valor': 'MORTGAGE'}, 'annual_inc': {'estrategia': 'media', 'valor': 74689.2425825}, 'verification_status': {'estrategia': 'moda', 'valor': 'Source Verified'}, 'loan_status': {'estrategia': 'moda', 'valor': 'Fully Paid'}, 'pymnt_plan': {'estrategia': 'moda', 'valor': 'n'}, 'desc': {'e

Imputación con la Mediana para Datos Numéricos:

Se eligió la mediana sobre la media para la imputación de datos numéricos debido a su resistencia a los valores atípicos. La mediana es una medida de tendencia central más robusta en presencia de valores extremos, lo cual es común en conjuntos de datos financieros y de préstamos. Esto garantiza que la imputación no sesgue los datos y refleje más fielmente la distribución original.
Imputación con la Moda para Datos Categóricos:

La moda es el enfoque estándar para imputar datos categóricos, ya que es la categoría más frecuentemente observada. Esto mantiene la consistencia y la validez de los datos categóricos, asegurando que la imputación no introduzca categorías no representativas.

### Codigo para salvar y cargar JSONs

In [27]:
with open('imputation_strategies.json', 'w') as f:
    json.dump(imputation_dict, f)

print("Archivo JSON guardado con éxito.")

Archivo JSON guardado con éxito.


In [24]:
def cargar_estrategias_imputacion(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

# Cargar el archivo JSON y almacenarlo en una variable
estrategias_imputacion = cargar_estrategias_imputacion('imputation_strategies.json')

# Mostrar las estrategias cargadas
print(estrategias_imputacion)

{'member_id': {'estrategia': 'all_nan', 'valor': None}, 'annual_inc_joint': {'estrategia': 'all_nan', 'valor': None}, 'dti_joint': {'estrategia': 'all_nan', 'valor': None}, 'verification_status_joint': {'estrategia': 'all_nan', 'valor': None}, 'open_acc_6m': {'estrategia': 'all_nan', 'valor': None}, 'open_act_il': {'estrategia': 'all_nan', 'valor': None}, 'open_il_12m': {'estrategia': 'all_nan', 'valor': None}, 'open_il_24m': {'estrategia': 'all_nan', 'valor': None}, 'mths_since_rcnt_il': {'estrategia': 'all_nan', 'valor': None}, 'total_bal_il': {'estrategia': 'all_nan', 'valor': None}, 'il_util': {'estrategia': 'all_nan', 'valor': None}, 'open_rv_12m': {'estrategia': 'all_nan', 'valor': None}, 'open_rv_24m': {'estrategia': 'all_nan', 'valor': None}, 'max_bal_bc': {'estrategia': 'all_nan', 'valor': None}, 'all_util': {'estrategia': 'all_nan', 'valor': None}, 'inq_fi': {'estrategia': 'all_nan', 'valor': None}, 'total_cu_tl': {'estrategia': 'all_nan', 'valor': None}, 'inq_last_12m': {'es